In [3]:
import requests
import pandas as pd
import csv
from bs4 import BeautifulSoup
import re

In [4]:
#stating url
url = 'https://www.shepherd.edu/cme/staff'

response = requests.get(url)
#check to see if successful
if response.status_code == 200:
    
    soup = BeautifulSoup(response.text, 'html.parser')

    faculty_data = []
    #looked at the source code to determine which class and location
    for faculty in soup.find_all('div', class_="inner"):
        faculty_info = {}
        #name
        name_elem = faculty.find('h2')
        faculty_info['Name'] = name_elem.text.strip() if name_elem else ''
        
        #title
        title = faculty.find('td')
        if title:
            faculty_info['Title'] = title.text.strip()
        else:
            faculty_info['Title'] = "Title not found"
       #email     
        email_link = faculty.find('a', href=True, text='Email')
        if email_link:
            faculty_info['Email'] = email_link['href'].replace('mailto:', '')
        else:
            faculty_info['Email'] = "Email not found"
        #phone since they all don't have a number you have to use re   
        phone_pattern = re.compile(r'\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b')
        td_elements = faculty.find_all('td')
        if len (td_elements)>=3:
                phone_match = phone_pattern.search(td_elements[2].text)
                 
                if phone_match:
                    faculty_info['Phone'] = phone_match.group()
                else:
                    faculty_info['Phone'] = "Phone not found"
        #putting the data into frame
        faculty_data.append(faculty_info)

    
    faculty_df = pd.DataFrame(faculty_data)

    # Write the data to a CSV file
    csv_file = 'faculty_info.csv'
    faculty_df.to_csv(csv_file, index=False)

    print(f"Faculty information has been scraped and saved to {csv_file}.")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


Faculty information has been scraped and saved to faculty_info.csv.
